Considering only the Ocean proximity, median income and housing median age for this one. Since they are the most contributing factors.

In [73]:
import pandas as pd
import numpy as np


In [157]:
df = pd.read_csv("housing.csv")

In [158]:
df["median_income"].isnull().values.any()

False

In [159]:
df.sample()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
18485,-121.57,37.0,18.0,7241.0,1225.0,4168.0,1138.0,4.5714,260300.0,INLAND


In [160]:
df.drop(["longitude", "latitude", "total_rooms", "total_bedrooms", "population"], axis=1, inplace=True)

In [161]:
df.sample()

,housing_median_age,households,median_income,median_house_value,ocean_proximity
20016,21.0,221.0,2.5893,63600.0,INLAND


In [162]:
df["cat_income"] = np.ceil(df["median_income"]/1.5)

In [163]:
df.sample()

,housing_median_age,households,median_income,median_house_value,ocean_proximity,cat_income
227,48.0,238.0,4.744,257300.0,NEAR BAY,4.0


In [164]:
df["cat_income"].value_counts()

3.0     7236
2.0     6581
4.0     3639
5.0     1423
1.0      822
6.0      532
7.0      189
8.0      105
9.0       50
11.0      49
10.0      14
Name: cat_income, dtype: int64

In [165]:
df = df[df["cat_income"]<9]

In [166]:
df["cat_income"].value_counts()

3.0    7236
2.0    6581
4.0    3639
5.0    1423
1.0     822
6.0     532
7.0     189
8.0     105
Name: cat_income, dtype: int64

In [167]:
df.sample()

,housing_median_age,households,median_income,median_house_value,ocean_proximity,cat_income
18230,17.0,180.0,3.4375,290600.0,NEAR BAY,3.0


In [168]:
df.shape

(20527, 6)

In [169]:
df["median_income"].isnull().values.any()

False

In [144]:
#df["cat_income"].where(df["cat_income"] <=5, 5, inplace=True)

In [170]:
df["cat_income"].value_counts()

3.0    7236
2.0    6581
4.0    3639
5.0    1423
1.0     822
6.0     532
7.0     189
8.0     105
Name: cat_income, dtype: int64

In [171]:
df = pd.get_dummies(df, prefix="ocean_proximity", drop_first=True)

In [172]:
df.sample()

,housing_median_age,households,median_income,median_house_value,cat_income,ocean_proximity_INLAND,ocean_proximity_ISLAND,ocean_proximity_NEAR BAY,ocean_proximity_NEAR OCEAN
6783,39.0,447.0,3.6406,278900.0,3.0,0,0,0,0


In [173]:
df["cat_income"].value_counts()/df.shape[0]

3.0    0.352511
2.0    0.320602
4.0    0.177279
5.0    0.069323
1.0    0.040045
6.0    0.025917
7.0    0.009207
8.0    0.005115
Name: cat_income, dtype: float64

In [174]:
df.isnull().values.any()

False

In [175]:
df["median_income"].isnull().values.any()

False

In [176]:
df["median_income"].fillna(df["median_income"].median(), inplace=True)

In [177]:
from sklearn.model_selection import StratifiedShuffleSplit
split = StratifiedShuffleSplit(n_splits=1, test_size=.2, random_state=42)

In [183]:
for train_index, test_index in split.split(df, df["cat_income"]):
    strat_train_set = df.iloc[train_index]
    strat_test_set = df.iloc[test_index]

In [184]:
df["median_income"].isnull().values.any()

False

In [185]:
df.shape

(20527, 9)

In [187]:
strat_train_set.isnull().values.any()

False

In [188]:
strat_train_set.shape

(16421, 9)

In [189]:
strat_train_set["median_income"].isnull().values.any()

False

In [190]:
strat_train_set["cat_income"].value_counts()/strat_train_set.shape[0]

3.0    0.352536
2.0    0.320626
4.0    0.177273
5.0    0.069302
1.0    0.040010
6.0    0.025942
7.0    0.009196
8.0    0.005115
Name: cat_income, dtype: float64

In [191]:
strat_test_set["cat_income"].value_counts()/strat_test_set.shape[0]

3.0    0.352411
2.0    0.320507
4.0    0.177302
5.0    0.069411
1.0    0.040185
6.0    0.025816
7.0    0.009255
8.0    0.005114
Name: cat_income, dtype: float64

In [192]:
strat_train_set.sample()

,housing_median_age,households,median_income,median_house_value,cat_income,ocean_proximity_INLAND,ocean_proximity_ISLAND,ocean_proximity_NEAR BAY,ocean_proximity_NEAR OCEAN
6708,45.0,181.0,2.3,55000.0,2.0,0,0,0,0


In [193]:
strat_train_set["median_income"].isnull().values.any()

False

In [194]:
arr = strat_train_set.iloc[:, 2].values

In [195]:
np.count_nonzero(arr)

16421

In [196]:
len(arr)

16421

In [197]:
y_train = strat_train_set.iloc[:, 3].values
y_test = strat_test_set.iloc[:, 3].values

In [198]:
strat_train_set = strat_train_set.drop(["median_house_value"], axis=1)
strat_test_set = strat_test_set.drop(["median_house_value"], axis=1)

In [199]:
x_train = strat_train_set.iloc[:,:].values
x_test = strat_test_set.iloc[:,:].values

In [200]:
from sklearn.linear_model import LinearRegression
regressor = LinearRegression()
regressor.fit(x_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [201]:
y_pred = regressor.predict(x_test)

In [202]:
from sklearn.metrics import mean_squared_error
lin_rmse = mean_squared_error(y_test, y_pred)
np.sqrt(lin_rmse)

71141.59751742371